In [19]:
from multiprocessing import Process, Queue
from threading import Thread

import time

reference :

1. [프로세스와 스레드](https://brunch.co.kr/@kd4/3)

### 프로세스(Process)와 스레드(Thread)

#### 1. 프로세스(Process)

운영체제로부터 자원을 할당받은 작업의 단위! 그래서 독립적


장점 : 
   * 메모리를 공유하지 않음
   * 코드 흐름이 명확함
   * 멀티코어의 장점을 쓸 수 있음
   * Shared Memory를 쓰지 않는 이상 동기화가 필요 없음
   * 자식 프로세스를 Interrupt/Kill할 수 있음

단점 : 
   * 메모리가 더 필요함


#### 2. 스레드(Thread)

프로세스가 할당받
은 자원을 이용하는 실행의 단위! 그래서 종속적

장점 : 
   * 메모리가 적게 필요함
   * 메모리를 공유함 - 서로 상태를 공유하기 쉬움
   * GIL을 이용해 병렬 처리가 가능
단점 :
   * interrupt/kill 할 수 없음
   * race condition이 발생할 수 있음

#### 3. GIL

인터프리터도 자원을 보호해야 한다. 

GIL의 장점

* 인터프리터의 구현이 쉬워진다.
* 가비지 컬렉터를 만들기도 좋다. 
* C/C++ 확장 모듈을 만들기에도 좋다. 

GIL의 단점

* CPU를 동시에 단 하나 밖에 사용하지 못한다.

In [21]:
# Start 부터 End까지 더하는 함수
def do_work(start, end, result):
    total = 0
    for i in range(start, end):
        total += i
    result.append(total)

### 쓰레드를 하나 이용했을 때

In [31]:
START, END = 0, 2000000
result = list()
th1 = Thread(target=do_work, args=(START, END, result))

start_time = time.time()

th1.start()
th1.join()

print("Seconds : {}".format(time.time() - start_time))

Seconds : 0.11690402030944824


In [32]:
print(result)

[1999999000000]


### 쓰레드를 두개 이용했을 때

In [33]:
START, END = 0, 2000000
result = list()
th1 = Thread(target=do_work, args=(START, END//2, result))
th2 = Thread(target=do_work, args=(END//2, END, result))

start_time = time.time()

th1.start()
th2.start()
th1.join()
th2.join()

print("Seconds : {}".format(time.time() - start_time))

Seconds : 0.1184530258178711


In [35]:
sum(result)

1999999000000

### 프로세스를 2개 이용했을 때

In [40]:
START, END = 0, 2000000
result = list()
pr1 = Process(target=do_work, args=(START, END//2, result))
pr2 = Process(target=do_work, args=(END//2, END, result))

start_time = time.time()

pr1.start()
pr2.start()
pr1.join()
pr2.join()

print("Seconds : {}".format(time.time() - start_time))

Seconds : 0.07749295234680176


In [42]:
print(result)

[]


...?

In [43]:
from multiprocessing import Queue

In [44]:
# Start 부터 End까지 더하는 함수
def do_work(start, end, result):
    total = 0
    for i in range(start, end):
        total += i
    result.put(total)

In [45]:
START, END = 0, 2000000
result = Queue()
pr1 = Process(target=do_work, args=(START, END//2, result))
pr2 = Process(target=do_work, args=(END//2, END, result))

start_time = time.time()

pr1.start()
pr2.start()
pr1.join()
pr2.join()

print("Seconds : {}".format(time.time() - start_time))

Seconds : 0.0784749984741211


In [54]:
total = 0
while not result.empty():
    
    tmp = result.get()
    print(tmp)
    total += tmp
print(total)

0


1999999000000

In [46]:
while result.get()